# Обрезка полей: обработка баз открыток

## Алгоритм

Используется три метода:
* **OpenCV** - сканнер документов на основе `OpenCV`
* **rembg** - библиотека для удаления фона с изображений 
* **алгоритмический подход** - разработан специально для обрезки открыток

Подробнее о методах с примерами работы - в специальных ноутбуках.

Метод OpenCV можно рассматривать как основной, а другие два метода - как вспомогательные.  
Обрезка происходит так:
1. Сначала все сканы обрезаются с помощью OpenCV. 
2. Затем программа автоматически разделяет результаты обрезки на успешные и провальные. Успешные результаты сохраняются в директорию `cut-success`, а провальные - в директорию `cut-fail`.
3. После этого сканы, на которых OpenCV не сработал, обрезаются заново с помощью rembg и алгоритмического подхода. Результаты также сохраняются в директорию `cut-fail`.

# Функции

In [14]:
from tqdm import tqdm
import requests
import os
import traceback

Функция для скачивания изображения из хранилища

In [16]:
def download(url, filename):
    photo_content = requests.get(url).content
    with open(filename, 'wb') as photo_file:
        photo_file.write(photo_content)

Функция для генерации названий файлов с результатами обрезки

In [37]:
def get_output_filenames(input_filename):
    name, extension = input_filename.split('.')
    opencv_filename = name + '-cut-opencv.' + extension
    rembg_filename = name + '-cut-rembg.' + extension
    alg_filename = name + '-cut-alg.' + extension
    return opencv_filename, rembg_filename, alg_filename

Импорты функций из скриптов

In [18]:
from crop_rembg import crop_rembg
from crop_opencv import crop_opencv
from crop_algorithmic import crop_algorithmic

"Верховная" функция:
1. Скачивает изображение
2. Обрезает OpenCV
3. Кладет в нужную директорию (успешная / провальная обрезка)
4. Если обрезка провальная, добавляет варианты обрезки rembg и алгоритмом Ильи

Исключения записываются в `exceptions.txt`.

In [50]:
def process_image(
    input_filename, 
    download_url,
    main_folder='pictures/', 
    success_subfolder='cut-success/', 
    fail_subfolder='cut-fail/'
    ):

    try:
        # download from storage
        download(download_url, main_folder + input_filename)
        # generate output filenames
        opencv_filename, rembg_filename, alg_filename = get_output_filenames(input_filename)
        
        # crop with OpenCV
        crop_successful = crop_opencv(main_folder + input_filename, main_folder + opencv_filename)
        
        # crop successful
        if crop_successful:
            # move files to successful crop subfolder
            os.rename(main_folder + input_filename, main_folder + success_subfolder + input_filename)
            os.rename(main_folder + opencv_filename, main_folder + success_subfolder + opencv_filename)
            return
        
        # crop failed
        # move files to failed crop subfolder
        os.rename(main_folder + input_filename, main_folder + fail_subfolder + input_filename)
        os.rename(main_folder + opencv_filename, main_folder + fail_subfolder + opencv_filename)
        # crop with rembg and algorithmic approach 
        # and save outputs to failed crop subfolder
        crop_rembg(main_folder + fail_subfolder + input_filename, main_folder + fail_subfolder + rembg_filename)
        crop_algorithmic(main_folder + fail_subfolder + input_filename, main_folder + fail_subfolder + alg_filename)
  
    except:
#         raise
        with open('exceptions.txt', 'a') as exceptions_file:
            exceptions_file.write('input_filename: ' + input_filename + '\n')
            exceptions_file.write(traceback.format_exc() + '\n\n\n')

# Обработка баз

## База 1
Чтение данных

In [15]:
import pandas as pd

In [16]:
df = pd.read_excel('db/Расшифровка открыток (основная база).xlsx')
df = df.rename(columns={' ': 'ID'})
df['ID'] = df['ID'].astype(int)
df.head()

,ID,Pic_url_1,Pic_url_2,Разрешение на публикацию,Источник,Была ли открытка отправлена по почте,"Название коллекции (если вы хотите подпись на сайте). Например, личная коллекция Василия Иванова.","Комментарий или метаинформация. Укажите тут любую метаинформацию про открытку, которую считаете важной. Например, какое-то знание или история, которую нельзя восстановить по тексту открытки.",Заливщик,Разметчик,...,Тег_3,Тег_4,Тег_5,Тег_6,Тег_7,Интересна ли для медиа/исследователей,Почему интересна,Id поста на сайте,Дата заливки,Оценка
0,0,https://forum.vgd.ru/file.php?fid=5476&key=0,0.0,Разрешаю публикацию,Форум VGD,Да,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,https://forum.vgd.ru/file.php?fid=5491&key=0,NaN,Разрешаю публикацию,Форум VGD,Не знаю,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,https://forum.vgd.ru/file.php?fid=5492&key=0,0.0,Разрешаю публикацию,Форум VGD,Не знаю,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,https://forum.vgd.ru/file.php?fid=20890&key=19...,0.0,Разрешаю публикацию,Форум VGD,Не знаю,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,https://forum.vgd.ru/file.php?fid=20894&key=16...,0.0,Разрешаю публикацию,Форум VGD,Не знаю,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df.shape

(7485, 58)

In [18]:
df['Проблема'].value_counts()

Отсутствует                         5581
Вотермарка                           962
Большие поля                         155
Низкое качество                       35
Скрыть информацию                     24
Требуется поворот                     21
Нет текста                            14
Две стороны на одном изображении       6
Name: Проблема, dtype: int64

Оставляем строки с проблемой "Большие поля"

In [19]:
df = df[df['Проблема'] == 'Большие поля']
df.shape

(155, 58)

In [20]:
df['Статус (распределено, в процессе, завершено, проблема)'].value_counts()

Готово к расшифровке                   109
Исключено                               30
Завершено                                8
Проверено                                6
В процессе                               1
Помощь модератора/ иностранный язык      1
Name: Статус (распределено, в процессе, завершено, проблема), dtype: int64

Убираем из рассмотрения уже исключенные открытки

In [21]:
df = df[df['Статус (распределено, в процессе, завершено, проблема)'] != 'Исключено']
df.shape

(125, 58)

Запуск обрезки

In [69]:
for ind, row in tqdm(df.iterrows(), total=df.shape[0]):
  postcard_id = row['ID']

  filename_1 = str(postcard_id) + '-1.jpg'
  filename_2 = str(postcard_id) + '-2.jpg'
  filename_1_cut = str(postcard_id) + '-1-cut.jpg'
  filename_2_cut = str(postcard_id) + '-2-cut.jpg'

  process_image(filename_1, filename_1_cut, row['Pic_url_1'])
  process_image(filename_2, filename_2_cut, row['Pic_url_2'])

100%|██████████| 155/155 [1:12:33<00:00, 28.09s/it] 


Упаковка результатов в архив

In [ ]:
!zip -r /content/pictures.zip /content/pictures

## База 2
Чтение данных

In [71]:
xls = pd.ExcelFile('/content/drive/MyDrive/Системный Блок/пишу тебе/открытки базы/Расшифровка открыток 2 (Ярмарка проектов).xlsx')
df = pd.read_excel(xls, 'Открытки')
df = df.rename(columns={'Unnamed: 0': 'ID'})
df['ID'] = df['ID'].astype(int)
df.head()

,ID,Pic_url_1,Pic_url_2,Разрешение на публикацию,Источник,Была ли открытка отправлена по почте,"Название коллекции (если вы хотите подпись на сайте). Например, личная коллекция Василия Иванова.","Комментарий или метаинформация. Укажите тут любую метаинформацию про открытку, которую считаете важной. Например, какое-то знание или история, которую нельзя восстановить по тексту открытки.",Заливщик,Разметчик,...,Тег_3,Тег_4,Тег_5,Тег_6,Тег_7,Интересна ли для медиа/исследователей,Почему интересна,Id поста на сайте,Дата заливки,Оценка
0,1,https://storage.yandexcloud.net/postcards-open...,https://storage.yandexcloud.net/postcards-open...,Разрешаю публикацию,Найдено в интернете,NaN,NaN,NaN,NaN,Атрощенко Екатерина (ЯП),...,Письма семье,NaN,NaN,NaN,NaN,NaN,NaN,26789.0,Thu Aug 25 2022 15:04:40 GMT-0400 (Восточная А...,5
1,2,https://storage.yandexcloud.net/postcards-open...,https://storage.yandexcloud.net/postcards-open...,Разрешаю публикацию,Найдено в интернете,NaN,NaN,NaN,NaN,Зыкова Мария (ЯП),...,Благодарность в тексте,Поцелуи из писем,Деньги,NaN,NaN,NaN,NaN,26786.0,Thu Aug 25 2022 15:04:40 GMT-0400 (Восточная А...,4
2,3,https://storage.yandexcloud.net/postcards-open...,https://storage.yandexcloud.net/postcards-open...,Разрешаю публикацию,Найдено в интернете,NaN,NaN,NaN,NaN,Залян Эдуард (ЯП),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26807.0,Thu Aug 25 2022 15:04:40 GMT-0400 (Восточная А...,5
3,4,https://storage.yandexcloud.net/postcards-open...,https://storage.yandexcloud.net/postcards-open...,Разрешаю публикацию,Найдено в интернете,NaN,NaN,NaN,NaN,Никитина Полина (ЯП),...,Загадочная открытка,NaN,NaN,NaN,NaN,NaN,NaN,26788.0,Thu Aug 25 2022 15:04:40 GMT-0400 (Восточная А...,4
4,5,https://storage.yandexcloud.net/postcards-open...,https://storage.yandexcloud.net/postcards-open...,Разрешаю публикацию,Найдено в интернете,NaN,NaN,NaN,NaN,Кошкина Мария (ЯП),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26804.0,Thu Aug 25 2022 15:04:40 GMT-0400 (Восточная А...,5


In [101]:
df.shape

(4968, 58)

In [72]:
df['Проблема'].value_counts()

Отсутствует                         4490
Большие поля (обе)                   219
Низкое качество                       84
Большие поля (лицевая)                84
Требуется поворот                     33
Нет текста                            15
Большие поля (оборотная)              12
Две стороны на одном изображении       1
Скрыть информацию                      1
Name: Проблема, dtype: int64

Отбираем открытки с проблемой "Большие поля"

In [74]:
df = df[df['Проблема'].isin(('Большие поля (обе)', 
                             'Большие поля (лицевая)', 'Большие поля (оборотная)'))]
df.shape

(315, 58)

In [75]:
df['Статус (распределено, в процессе, завершено, проблема)'].value_counts()

Проверено                                         247
Завершено                                          30
Готово к расшифровке                               18
Распределено                                       11
Помощь модератора/ не разобрать слова в тексте      5
Исключено/ трудный почерк                           2
Исключено                                           1
Помощь модератора/ иностранный язык                 1
Name: Статус (распределено, в процессе, завершено, проблема), dtype: int64

Выкидываем уже исключенные

In [76]:
df = df[~df['Статус (распределено, в процессе, завершено, проблема)'].isin((
    'Исключено/ трудный почерк', 'Исключено'))]
df.shape

(312, 58)

Запуск обрезки

In [77]:
for ind, row in tqdm(df.iterrows(), total=df.shape[0]):
  postcard_id = row['ID']
  
  if row['Проблема'] in ('Большие поля (лицевая)', 'Большие поля (обе)'):
    filename_1 = str(postcard_id) + '-1.jpg'
    filename_1_cut = str(postcard_id) + '-1-cut.jpg'
    process_image(filename_1, filename_1_cut, row['Pic_url_1'])
  
  if row['Проблема'] in ('Большие поля (оборотная)', 'Большие поля (обе)'):
    filename_2 = str(postcard_id) + '-2.jpg'
    filename_2_cut = str(postcard_id) + '-2-cut.jpg'
    process_image(filename_2, filename_2_cut, row['Pic_url_2'])

100%|██████████| 312/312 [1:19:56<00:00, 15.37s/it]


In [ ]:
!zip -r /content/pictures.zip /content/pictures

## База 3
Чтение данных

In [31]:
xls = pd.ExcelFile('db/Расшифровка открыток 3 (Ярмарка проектов).xlsx')
df = pd.read_excel(xls, 'Открытки')
df = df.rename(columns={'Unnamed: 0': 'ID'})
df = df[~df['ID'].isna()]
df['ID'] = df['ID'].astype(int)
df.head()

,ID,Pic_url_1,Pic_url_2,Разрешение на публикацию,Источник,Была ли открытка отправлена по почте,"Название коллекции (если вы хотите подпись на сайте). Например, коллекция Василия Иванова.","Комментарий или метаинформация. Укажите тут любую метаинформацию про открытку, которую считаете важной. Например, какое-то знание или история, которую нельзя восстановить по тексту открытки.",Заливщик,Разметчик,...,Тег_3,Тег_4,Тег_5,Тег_6,Тег_7,Интересна ли для медиа/исследователей,Почему интересна,Id поста на сайте,Дата заливки,Оценка
0,5028,https://storage.yandexcloud.net/postcards-open...,https://storage.yandexcloud.net/postcards-open...,Разрешаю публикацию,Найдено в интернете,NaN,NaN,NaN,NaN,Анна Макарова,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,5096,https://storage.yandexcloud.net/postcards-open...,https://storage.yandexcloud.net/postcards-open...,Разрешаю публикацию,Найдено в интернете,NaN,NaN,NaN,NaN,Анна Макарова,...,Письма семье,Написано о детях,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,4980,https://storage.yandexcloud.net/postcards-open...,https://storage.yandexcloud.net/postcards-open...,Разрешаю публикацию,Найдено в интернете,NaN,NaN,NaN,NaN,"Бачинина Яна (ЯП4, 09-10.22)",...,Социальное,Житейское,Загадочная открытка,NaN,NaN,NaN,NaN,NaN,NaN,5
3,5038,https://storage.yandexcloud.net/postcards-open...,https://storage.yandexcloud.net/postcards-open...,Разрешаю публикацию,Найдено в интернете,NaN,NaN,NaN,NaN,"Бачинина Яна (ЯП4, 09-10.22)",...,Письма от семьи,Житейское,Новости,Дурные вести,пасха,NaN,NaN,NaN,NaN,4
4,5073,https://storage.yandexcloud.net/postcards-open...,https://storage.yandexcloud.net/postcards-open...,Разрешаю публикацию,Найдено в интернете,NaN,NaN,NaN,NaN,"Бенедиктова Полина (ЯП4, 09-10.22)",...,Просьба,Житейское,Письма отцу,NaN,Из путешествия,NaN,NaN,NaN,NaN,4


In [32]:
df.shape

(10453, 58)

In [33]:
df['Проблема'].value_counts()

Отсутствует                         7698
Большие поля (обе)                  2589
Низкое качество                       99
Нет текста                             9
Требуется поворот                      7
Большие поля (оборотная)               4
Обрезать небольшую полоску снизу       1
Большие поля (лицевая)                 1
Скрыть информацию                      1
Две стороны на одном изображении       1
иностранный язык                       1
Name: Проблема, dtype: int64

Отбираем открытки с проблемой "Большие поля"

In [34]:
df = df[df['Проблема'].isin(['Большие поля (обе)', 'Большие поля (оборотная)', 'Большие поля (лицевая)'])]
df.shape

(2594, 58)

In [35]:
df['Статус (распределено, в процессе, завершено, проблема)'].value_counts()

Завершено                                                 1468
Проверено                                                  886
Распределено                                               208
Помощь модератора/ не разобрать слова в тексте              12
Исключено                                                    8
Помощь модератора/ иностранный язык                          4
В процессе                                                   4
Исключено/ трудный почерк                                    2
Помощь модератора/ информация об изображении, открытке       1
Готово к расшифровке                                         1
Name: Статус (распределено, в процессе, завершено, проблема), dtype: int64

Выкидываем уже исключенные

In [36]:
df = df[~df['Статус (распределено, в процессе, завершено, проблема)'].isin(['Исключено', 'Исключено/ трудный почерк'])]
df.shape

(2584, 58)

Запуск обрезки

In [51]:
for ind, row in tqdm(df.iterrows(), total=df.shape[0]):
    postcard_id = row['ID']

    if row['Проблема'] in ('Большие поля (лицевая)', 'Большие поля (обе)'):
        filename_front = str(postcard_id) + '-1.jpg'
        process_image(filename_front, row['Pic_url_1'], main_folder='pictures 3/')

    if row['Проблема'] in ('Большие поля (оборотная)', 'Большие поля (обе)'):
        filename_back = str(postcard_id) + '-2.jpg'
        process_image(filename_back, row['Pic_url_2'], main_folder='pictures 3/')

100%|██████████| 2584/2584 [13:02:35<00:00, 18.17s/it]    


# Таблица с метаинформацией
Создаем таблицу с информацией о результатах обрезки  
https://docs.google.com/spreadsheets/d/1kOEF__dBfsv2DG-IQid1o9KWkIzNUfk5/edit?usp=sharing&ouid=109949538838834877697&rtpof=true&sd=true

Инструкция для ручной корректировки данных  
https://docs.google.com/document/d/1ZZGgGRIv78tdaEh5h00GI_tuZ5vckQJWoGe9tmDNGgM/edit?usp=sharing

## Функции

Лист о директории cut-success

In [91]:
def get_cut_success_df(directory_str):
    directory = os.fsencode(directory_str)
    orig = True
    df_cut_success = []

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if orig:
            name, extension = filename.split('.')
            name = name.split('-')
            postcard_id, pic = int(name[0]), int(name[1])
            cur_item = [postcard_id, pic, filename]
        else:
            cur_item.append(filename)
            df_cut_success.append(tuple(cur_item))    
        orig = not orig
    
    df_cut_success = pd.DataFrame(df_cut_success, columns=['ID', 'Pic', 'crop_opencv', 'orig_file'])
    df_cut_success = df_cut_success[['ID', 'Pic', 'orig_file', 'crop_opencv']]
    df_cut_success['opencv_status'] = 'success'
    df_cut_success['comments'] = ''
    df_cut_success = df_cut_success.sort_values(by='ID')
    return df_cut_success

Лист о директории cut-fail

In [92]:
def get_cut_fail_df(directory_str):
    directory = os.fsencode(directory_str)
    df_cut_fail = []

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        name, extension = filename.split('.')
        name = name.split('-')
        postcard_id, pic = int(name[0]), int(name[1])
        if name[-1] == 'alg':
            cur_item = [postcard_id, pic, filename]
        elif name[-1] == 'opencv':
            cur_item.append(filename)
        elif name[-1] == 'rembg':
            cur_item.append(filename)
        else:
            cur_item.append(filename)
            df_cut_fail.append(tuple(cur_item)) 
            
    df_cut_fail = pd.DataFrame(df_cut_fail, 
                               columns=['ID', 'Pic', 'crop_alg', 'crop_opencv', 'crop_rembg', 'orig_file'])
    df_cut_fail = df_cut_fail[['ID', 'Pic', 'orig_file', 'crop_opencv', 'crop_rembg', 'crop_alg']]
    df_cut_fail['crop_result'] = 'rembg'
    df_cut_fail['comments'] = ''
    df_cut_fail = df_cut_fail.sort_values(by='ID')
    return df_cut_fail

## База 1

Переименовываем файлы (добавляем хвост -opencv) --- ***исторический код, при использовании верховной функции `process_image` не нужен***

In [60]:
directory = os.fsencode('pictures 1/cut-success/')
for file in os.listdir(directory):
    old_filename = os.fsdecode(file)
    if old_filename.endswith('-cut.jpg'):
        new_filename = old_filename.split('.')[0] + '-opencv.' + old_filename.split('.')[1]
        os.rename('pictures 1/cut-success/' + old_filename, 'pictures 1/cut-success/' + new_filename)

In [32]:
directory = os.fsencode('pictures 1/cut-fail/')
for file in os.listdir(directory):
    old_filename = os.fsdecode(file)
    if old_filename.endswith('-cut.jpg'):
        new_filename = old_filename.split('.')[0] + '-opencv.' + old_filename.split('.')[1]
        os.rename('pictures 1/cut-fail/' + old_filename, 'pictures 1/cut-fail/' + new_filename)

Генерируем лист cut-success

In [93]:
base_1_cut_success = get_cut_success_df('pictures 1/cut-success/')
base_1_cut_success.head()

,ID,Pic,orig_file,crop_opencv,opencv_status,comments
250,966,1,966-1.jpg,966-1-cut-opencv.jpg,success,
0,1649,1,1649-1.jpg,1649-1-cut-opencv.jpg,success,
1,2102,2,2102-2.jpg,2102-2-cut-opencv.jpg,success,
2,2150,1,2150-1.jpg,2150-1-cut-opencv.jpg,success,
3,2150,2,2150-2.jpg,2150-2-cut-opencv.jpg,success,


In [96]:
base_1_cut_success.shape

(251, 6)

In [97]:
base_1_cut_success['orig_file'].nunique()

251

Генерируем лист cut-fail

In [98]:
base_1_cut_fail = get_cut_fail_df('pictures 1/cut-fail/')
base_1_cut_fail.head()

,ID,Pic,orig_file,crop_opencv,crop_rembg,crop_alg,crop_result,comments
55,952,2,952-2.jpg,952-2-cut-opencv.jpg,952-2-cut-rembg.jpg,952-2-cut-alg.jpg,rembg,
54,952,1,952-1.jpg,952-1-cut-opencv.jpg,952-1-cut-rembg.jpg,952-1-cut-alg.jpg,rembg,
0,1024,1,1024-1.jpg,1024-1-cut-opencv.jpg,1024-1-cut-rembg.jpg,1024-1-cut-alg.jpg,rembg,
1,1024,2,1024-2.jpg,1024-2-cut-opencv.jpg,1024-2-cut-rembg.jpg,1024-2-cut-alg.jpg,rembg,
2,2102,1,2102-1.jpg,2102-1-cut-opencv.jpg,2102-1-cut-rembg.jpg,2102-1-cut-alg.jpg,rembg,


In [99]:
base_1_cut_fail.shape

(56, 8)

## База 2

Переименовываем файлы (добавляем хвост -opencv) --- ***исторический код, при использовании верховной функции `process_image` не нужен***

In [81]:
directory = os.fsencode('pictures 2/cut-success/')
for file in os.listdir(directory):
    old_filename = os.fsdecode(file)
    if old_filename.endswith('-cut.jpg'):
        new_filename = old_filename.split('.')[0] + '-opencv.' + old_filename.split('.')[1]
        os.rename('pictures 2/cut-success/' + old_filename, 'pictures 2/cut-success/' + new_filename)

In [82]:
directory = os.fsencode('pictures 2/cut-fail/')
for file in os.listdir(directory):
    old_filename = os.fsdecode(file)
    if old_filename.endswith('-cut.jpg'):
        new_filename = old_filename.split('.')[0] + '-opencv.' + old_filename.split('.')[1]
        os.rename('pictures 2/cut-fail/' + old_filename, 'pictures 2/cut-fail/' + new_filename)

Генерируем лист cut-success

In [101]:
base_2_cut_success = get_cut_success_df('pictures 2/cut-success/')
base_2_cut_success.head()

,ID,Pic,orig_file,crop_opencv,opencv_status,comments
263,57,1,57-1.jpg,57-1-cut-opencv.jpg,success,
264,57,2,57-2.jpg,57-2-cut-opencv.jpg,success,
6,148,1,148-1.jpg,148-1-cut-opencv.jpg,success,
7,148,2,148-2.jpg,148-2-cut-opencv.jpg,success,
9,177,2,177-2.jpg,177-2-cut-opencv.jpg,success,


In [102]:
base_2_cut_success.shape

(325, 6)

Генерируем лист cut-fail

In [103]:
base_2_cut_fail = get_cut_fail_df('pictures 2/cut-fail/')
base_2_cut_fail.head()

,ID,Pic,orig_file,crop_opencv,crop_rembg,crop_alg,crop_result,comments
31,177,1,177-1.jpg,177-1-cut-opencv.jpg,177-1-cut-rembg.jpg,177-1-cut-alg.jpg,rembg,
32,182,1,182-1.jpg,182-1-cut-opencv.jpg,182-1-cut-rembg.jpg,182-1-cut-alg.jpg,rembg,
34,198,1,198-1.jpg,198-1-cut-opencv.jpg,198-1-cut-rembg.jpg,198-1-cut-alg.jpg,rembg,
35,207,1,207-1.jpg,207-1-cut-opencv.jpg,207-1-cut-rembg.jpg,207-1-cut-alg.jpg,rembg,
36,209,1,209-1.jpg,209-1-cut-opencv.jpg,209-1-cut-rembg.jpg,209-1-cut-alg.jpg,rembg,


In [104]:
base_2_cut_fail.shape

(266, 8)

## База 3

Генерируем лист cut-success

In [105]:
base_3_cut_success = get_cut_success_df('pictures 3/cut-success/')
base_3_cut_success.head()

,ID,Pic,orig_file,crop_opencv,opencv_status,comments
3232,4974,2,4974-2.jpg,4974-2-cut-opencv.jpg,success,
3231,4974,1,4974-1.jpg,4974-1-cut-opencv.jpg,success,
3233,4975,2,4975-2.jpg,4975-2-cut-opencv.jpg,success,
3234,4977,1,4977-1.jpg,4977-1-cut-opencv.jpg,success,
3235,4977,2,4977-2.jpg,4977-2-cut-opencv.jpg,success,


In [107]:
base_3_cut_success.shape

(3256, 6)

Генерируем лист cut-fail

In [108]:
base_3_cut_fail = get_cut_fail_df('pictures 3/cut-fail/')
base_3_cut_fail.head()

,ID,Pic,orig_file,crop_opencv,crop_rembg,crop_alg,crop_result,comments
1896,4975,1,4975-1.jpg,4975-1-cut-opencv.jpg,4975-1-cut-rembg.jpg,4975-1-cut-alg.jpg,rembg,
1897,4987,1,4987-1.jpg,4987-1-cut-opencv.jpg,4987-1-cut-rembg.jpg,4987-1-cut-alg.jpg,rembg,
1898,5016,1,5016-1.jpg,5016-1-cut-opencv.jpg,5016-1-cut-rembg.jpg,5016-1-cut-alg.jpg,rembg,
1900,5110,2,5110-2.jpg,5110-2-cut-opencv.jpg,5110-2-cut-rembg.jpg,5110-2-cut-alg.jpg,rembg,
1899,5110,1,5110-1.jpg,5110-1-cut-opencv.jpg,5110-1-cut-rembg.jpg,5110-1-cut-alg.jpg,rembg,


In [109]:
base_3_cut_fail.shape

(1907, 8)

## Собираем единый файл excel

In [112]:
writer = pd.ExcelWriter('Обрезка открыток метаинформация.xlsx', engine='openpyxl')
base_1_cut_success.to_excel(writer, sheet_name='База 1 cut-success', index=False)
base_1_cut_fail.to_excel(writer, sheet_name='База 1 cut-fail', index=False)
base_2_cut_success.to_excel(writer, sheet_name='База 2 cut-success', index=False)
base_2_cut_fail.to_excel(writer, sheet_name='База 2 cut-fail', index=False)
base_3_cut_success.to_excel(writer, sheet_name='База 3 cut-success', index=False)
base_3_cut_fail.to_excel(writer, sheet_name='База 3 cut-fail', index=False)
writer.save()

# Рабочие таблицы для волонтеров
Задача:
* привести все директории к виду исходник - вариант обрезки
* переименовать файлы в директориях: 477-1.jpg, 477-1-crop.jpg

## Обработка директории cut-success

In [99]:
import os
import pandas as pd
import shutil

In [65]:
def cut_success_to_crop_opencv(cut_success_dir_str, crop_opencv_dir_str):
    cut_success_dir = os.fsencode(cut_success_dir_str)
    df_crop_opencv = []

    for file in reversed(sorted(os.listdir(cut_success_dir))):
        filename = os.fsdecode(file)
        if '-cut-opencv' in filename:
            new_filename = '-crop'.join(filename.split('-cut-opencv'))
            os.rename(cut_success_dir_str + filename, crop_opencv_dir_str + new_filename)
            cur_item.append(new_filename)
            df_crop_opencv.append(tuple(cur_item))
        else:
            os.rename(cut_success_dir_str + filename, crop_opencv_dir_str + filename)
            name, extension = filename.split('.')
            name = name.split('-')
            postcard_id, pic = int(name[0]), int(name[1])
            cur_item = [postcard_id, pic, filename]
    
    df_crop_opencv = pd.DataFrame(df_crop_opencv, columns=['ID', 'Pic', 'Название файла', 'Результат обрезки'])
    df_crop_opencv['Статус'] = ''
    df_crop_opencv['Комментарий'] = ''
    df_crop_opencv = df_crop_opencv.sort_values(by='ID')
    return df_crop_opencv

In [73]:
base_1_crop_opencv = cut_success_to_crop_opencv('pictures 1/cut-success/', 'pictures 1/crop-opencv/')
base_1_crop_opencv.head()

,ID,Pic,Название файла,Результат обрезки,Статус,Комментарий
0,966,1,966-1.jpg,1649-1-crop.jpg,,
1,1649,1,1649-1.jpg,2102-2-crop.jpg,,
2,2102,2,2102-2.jpg,2150-1-crop.jpg,,
3,2150,1,2150-1.jpg,2150-2-crop.jpg,,
4,2150,2,2150-2.jpg,2180-1-crop.jpg,,


In [57]:
base_1_crop_opencv.shape

(251, 6)

In [69]:
base_2_crop_opencv = cut_success_to_crop_opencv('pictures 2/cut-success/', 'pictures 2/crop-opencv/')
base_2_crop_opencv.head()

,ID,Pic,Название файла,Результат обрезки,Статус,Комментарий
60,57,2,57-2.jpg,57-2-crop.jpg,,
61,57,1,57-1.jpg,57-1-crop.jpg,,
317,148,2,148-2.jpg,148-2-crop.jpg,,
318,148,1,148-1.jpg,148-1-crop.jpg,,
315,177,2,177-2.jpg,177-2-crop.jpg,,


In [64]:
base_2_crop_opencv.shape

(325, 6)

In [85]:
base_3_crop_opencv = cut_success_to_crop_opencv('pictures 3/cut-success/', 'pictures 3/crop-opencv/')
base_3_crop_opencv.head()

,ID,Pic,Название файла,Результат обрезки,Статус,Комментарий
3233,4974,2,4974-2.jpg,4975-2-crop.jpg,,
3232,4974,1,4974-1.jpg,4974-2-crop.jpg,,
3234,4975,2,4975-2.jpg,4977-1-crop.jpg,,
3236,4977,2,4977-2.jpg,4987-2-crop.jpg,,
3235,4977,1,4977-1.jpg,4977-2-crop.jpg,,


In [86]:
base_3_crop_opencv.shape

(3256, 6)

## Обработка директории cut-fail

In [117]:
def cut_fail_to_crop_opencv(cut_fail_dir_str, crop_opencv_dir_str, crop_rembg_dir_str, crop_alg_dir_str):
    cut_fail_dir = os.fsencode(cut_fail_dir_str)
    os.mkdir(crop_opencv_dir_str)
    os.mkdir(crop_rembg_dir_str)
    os.mkdir(crop_alg_dir_str)
    df_crop = []

    for file in reversed(sorted(os.listdir(cut_fail_dir_str))):
        filename = os.fsdecode(file)
        if 'rembg' in filename:
            new_filename = '-crop'.join(filename.split('-cut-rembg'))
            os.rename(cut_fail_dir_str + filename, crop_rembg_dir_str + new_filename)
            df_crop.append([postcard_id, pic, filename_orig, new_filename])
        
        elif 'opencv' in filename:
            new_filename = '-crop'.join(filename.split('-cut-opencv'))
            os.rename(cut_fail_dir_str + filename, crop_opencv_dir_str + new_filename)
            
        elif 'alg' in filename:
            new_filename = '-crop'.join(filename.split('-cut-alg'))
            os.rename(cut_fail_dir_str + filename, crop_alg_dir_str + new_filename)
        
        else:
            name, extension = filename.split('.')
            name = name.split('-')
            postcard_id, pic = int(name[0]), int(name[1])
            filename_orig = filename
            shutil.copyfile(cut_fail_dir_str + filename, crop_opencv_dir_str + filename)
            shutil.copyfile(cut_fail_dir_str + filename, crop_rembg_dir_str + filename)
            shutil.copyfile(cut_fail_dir_str + filename, crop_alg_dir_str + filename)
            
    df_crop = pd.DataFrame(df_crop, columns=['ID', 'Pic', 'Название файла', 'Результат обрезки'])
    df_crop['Статус'] = ''
    df_crop['Комментарий'] = ''
    df_crop = df_crop.sort_values(by='ID')
    return df_crop

In [107]:
base_1_fail_crop = cut_fail_to_crop_opencv(
    'pictures 1/cut-fail/',
    'pictures 1/crop-opencv-fail/',
    'pictures 1/crop-rembg/',
    'pictures 1/crop-alg/'
)
base_1_fail_crop.head()

,ID,Pic,Название файла,Результат обрезки,Статус,Комментарий
0,952,2,952-2.jpg,952-2-crop.jpg,,
1,952,1,952-1.jpg,952-1-crop.jpg,,
54,1024,2,1024-2.jpg,1024-2-crop.jpg,,
55,1024,1,1024-1.jpg,1024-1-crop.jpg,,
53,2102,1,2102-1.jpg,2102-1-crop.jpg,,


In [121]:
base_1_fail_crop.shape

(56, 6)

In [ ]:
base_2_fail_crop = cut_fail_to_crop_opencv(
    'pictures 2/cut-fail/',
    'pictures 2/crop-opencv-fail/',
    'pictures 2/crop-rembg/',
    'pictures 2/crop-alg/'
)

In [115]:
base_2_fail_crop = base_2_fail_crop.drop_duplicates()
base_2_fail_crop.head()

,ID,Pic,Название файла,Результат обрезки,Статус,Комментарий
703,177,1,177-1.jpg,177-1-crop.jpg,,
701,182,1,182-1.jpg,182-1-crop.jpg,,
695,198,1,198-1.jpg,198-1-crop.jpg,,
692,207,1,207-1.jpg,207-1-crop.jpg,,
689,209,1,209-1.jpg,209-1-crop.jpg,,


In [116]:
base_2_fail_crop.shape

(266, 6)

In [118]:
base_3_fail_crop = cut_fail_to_crop_opencv(
    'pictures 3/cut-fail/',
    'pictures 3/crop-opencv-fail/',
    'pictures 3/crop-rembg/',
    'pictures 3/crop-alg/'
)
base_3_fail_crop.head()

,ID,Pic,Название файла,Результат обрезки,Статус,Комментарий
10,4975,1,4975-1.jpg,4975-1-crop.jpg,,
9,4987,1,4987-1.jpg,4987-1-crop.jpg,,
8,5016,1,5016-1.jpg,5016-1-crop.jpg,,
6,5110,2,5110-2.jpg,5110-2-crop.jpg,,
7,5110,1,5110-1.jpg,5110-1-crop.jpg,,


In [119]:
base_3_fail_crop.shape

(1907, 6)

## Завершающие шаги форматирования таблиц и папок
Объединяем таблицы по opencv вместе (из success и fail)

In [120]:
base_1_crop_opencv = pd.concat([base_1_crop_opencv, base_1_fail_crop], sort=True)
base_2_crop_opencv = pd.concat([base_2_crop_opencv, base_2_fail_crop], sort=True)
base_3_crop_opencv = pd.concat([base_3_crop_opencv, base_3_fail_crop], sort=True)
base_1_crop_opencv.shape, base_2_crop_opencv.shape, base_3_crop_opencv.shape

((307, 6), (591, 6), (5163, 6))

Записываем в один документ excel

In [122]:
writer = pd.ExcelWriter('Обрезка открыток рабочая таблица для волонтеров 2.xlsx', engine='openpyxl')

base_1_crop_opencv.to_excel(writer, sheet_name='База 1 метод OpenCV', index=False)
base_2_crop_opencv.to_excel(writer, sheet_name='База 2 метод OpenCV', index=False)
base_3_crop_opencv.to_excel(writer, sheet_name='База 3 метод OpenCV', index=False)

base_1_fail_crop.to_excel(writer, sheet_name='База 1 метод rembg', index=False)
base_2_fail_crop.to_excel(writer, sheet_name='База 2 метод rembg', index=False)
base_3_fail_crop.to_excel(writer, sheet_name='База 3 метод rembg', index=False)

base_1_fail_crop.to_excel(writer, sheet_name='База 1 метод algorithm', index=False)
base_2_fail_crop.to_excel(writer, sheet_name='База 2 метод algorithm', index=False)
base_3_fail_crop.to_excel(writer, sheet_name='База 3 метод algorithm', index=False)

writer.save()

Объединяем папки по opencv

In [124]:
for file in os.listdir('pictures 1/crop-opencv-fail/'):
    filename = os.fsdecode(file)
    os.rename('pictures 1/crop-opencv-fail/' + filename, 'pictures 1/crop-opencv/' + filename)

In [127]:
len(os.listdir('pictures 1/crop-opencv/')) // 2

307

In [128]:
for file in os.listdir('pictures 2/crop-opencv-fail/'):
    filename = os.fsdecode(file)
    os.rename('pictures 2/crop-opencv-fail/' + filename, 'pictures 2/crop-opencv/' + filename)
len(os.listdir('pictures 2/crop-opencv/')) // 2

591

In [129]:
for file in os.listdir('pictures 3/crop-opencv-fail/'):
    filename = os.fsdecode(file)
    os.rename('pictures 3/crop-opencv-fail/' + filename, 'pictures 3/crop-opencv/' + filename)
len(os.listdir('pictures 3/crop-opencv/')) // 2

5163

Удаляем папки cut-fail (в них остались копии исходников, они больше не нужны)

In [130]:
shutil.rmtree('pictures 1/cut-fail/')
shutil.rmtree('pictures 2/cut-fail/')
shutil.rmtree('pictures 3/cut-fail/')

# Точность метода

## Точность сканнера OpenCV
Предварительная оценка на основе автоматической сортировки на success и fail

In [113]:
success_count = base_1_cut_success.shape[0] + base_2_cut_success.shape[0] + base_3_cut_success.shape[0]
fail_count = base_1_cut_fail.shape[0] + base_2_cut_fail.shape[0] + base_3_cut_fail.shape[0]
success_count / (success_count + fail_count)

0.6322389044712093